In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [12]:
from dotenv import load_dotenv
load_dotenv()

True

# Tool Creation

In [4]:
# tool create

@tool
def multiply(a:int, b:int) -> int:
    """Given 2 numbers a and b this tool returns their product"""
    return a*b

In [5]:
print(multiply.invoke({'a':2,'b':3}))

6


In [6]:
multiply.name

'multiply'

In [7]:
multiply.description

'Given 2 numbers a and b this tool returns their product'

In [8]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

# Tool Binding

In [13]:
llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash')

In [14]:
llm_with_tools = llm.bind_tools([multiply])

In [15]:
llm_with_tools.invoke("Hi how are you")

AIMessage(content='I am doing well, thank you for asking. How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--83b5b208-c0b2-411b-8e85-c3c08bb1e0e9-0')

# Tool Calling

In [18]:
result = llm_with_tools.invoke("Can you multiply 3 with 10")
print(result)

content='' additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 10.0}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--4554ef63-b1ce-484f-90aa-d8bd0c386d4f-0' tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 10.0}, 'id': '48cd6021-bb21-4911-8713-f8975990b48c', 'type': 'tool_call'}]


In [19]:
result.tool_calls # List of all the tools which can be used

[{'name': 'multiply',
  'args': {'a': 3.0, 'b': 10.0},
  'id': '48cd6021-bb21-4911-8713-f8975990b48c',
  'type': 'tool_call'}]

In [20]:
result.tool_calls[0] # The first tool which can be usedd

{'name': 'multiply',
 'args': {'a': 3.0, 'b': 10.0},
 'id': '48cd6021-bb21-4911-8713-f8975990b48c',
 'type': 'tool_call'}

# Tool Execution

In [21]:
# If we pass the args then we will get the result
multiply.invoke(result.tool_calls[0]['args'])

30

In [22]:
# If we pass the whole tool call then we get ToolMessage as result
multiply.invoke(result.tool_calls[0])

ToolMessage(content='30', name='multiply', tool_call_id='48cd6021-bb21-4911-8713-f8975990b48c')

# Sample

In [36]:
query = HumanMessage("Can you multiply 3 and 10")

In [37]:
messages = [query]

In [38]:
messages

[HumanMessage(content='Can you multiply 3 and 10', additional_kwargs={}, response_metadata={})]

In [ ]:
result = llm_with_tools.invoke(messages)

In [40]:
messages.append(result)

In [41]:
messages

[HumanMessage(content='Can you multiply 3 and 10', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--a6abab8c-97c2-4ead-a9cc-1f85c20d8cd0-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 10.0}, 'id': '54ead17d-c48d-48a0-b3a2-2691ae8a5af3', 'type': 'tool_call'}])]

In [42]:
tool_result = multiply.invoke(result.tool_calls[0])

In [43]:
messages.append(tool_result)

In [44]:
messages

[HumanMessage(content='Can you multiply 3 and 10', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--a6abab8c-97c2-4ead-a9cc-1f85c20d8cd0-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 10.0}, 'id': '54ead17d-c48d-48a0-b3a2-2691ae8a5af3', 'type': 'tool_call'}]),
 ToolMessage(content='30', name='multiply', tool_call_id='54ead17d-c48d-48a0-b3a2-2691ae8a5af3')]

In [45]:
llm_with_tools.invoke(messages)

AIMessage(content='3 multiplied by 10 is 30.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--60587433-de90-4891-befa-7968967c074d-0')

In [46]:
llm_with_tools.invoke(messages).content

'3 multiplied by 10 is 30.'